In [ ]:
import cdflib
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import os

In [ ]:
def match_epochs(raw_data, raw_epochs, label_epochs, labels):
    X, y = [], []
    for le, lab in zip(label_epochs, labels):
        idx = np.argmin(np.abs(raw_epochs - le))
        X.append(raw_data[idx])
        y.append(lab)
    return np.array(X), np.array(y)

def preprocess_distributions(X):
    X = X.copy()
    X[X == 0] = np.min(X[X > 0])
    X = np.log10(X)
    X = (X - X.min(axis=(1,2,3), keepdims=True)) / \
        (X.max(axis=(1,2,3), keepdims=True) - X.min(axis=(1,2,3), keepdims=True))
    return X[..., np.newaxis]

In [ ]:
def build_mms_cnn(input_shape=(32, 16, 32, 1), num_classes=4):
    """
    3D CNN for MMS ion distribution classification.
    
    input_shape : tuple
        Shape of the input sample (32 energy, 16 theta, 32 phi, 1 channel).
    num_classes : int
        Number of output classes (default = 4: SW, IF, MSH, MSP).
    
    Returns:
        A compiled Keras model.
    """
    model = models.Sequential([
        # First 3D convolution
        layers.Conv3D(
            filters=32, kernel_size=(5, 3, 5), strides=(2, 1, 2),
            activation='relu', padding='valid',
            input_shape=input_shape
        ),
        
        # Second 3D convolution
        layers.Conv3D(
            filters=32, kernel_size=(3, 3, 3), strides=(1, 1, 1),
            activation='relu', padding='valid'
        ),
        
        # Max pooling
        layers.MaxPooling3D(pool_size=(2, 2, 2)),
        
        # Flatten for dense layers
        layers.Flatten(),
        
        # Dense hidden layer
        layers.Dense(128, activation='relu'),
        
        # Output layer
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# trying different approach 


In [ ]:


# -------------------------------
# Your helper functions
# -------------------------------

def match_epochs(raw_data, raw_epochs, label_epochs, labels):
    """Match raw_data samples with label epochs"""
    # Assuming raw_epochs and label_epochs are aligned or nearly aligned
    # Nearest-neighbor match
    matched_X, matched_y = [], []
    raw_epochs = np.array(raw_epochs)
    for le, lab in zip(label_epochs, labels):
        idx = np.argmin(np.abs(raw_epochs - le))
        matched_X.append(raw_data[idx])
        matched_y.append(lab)
    return np.array(matched_X), np.array(matched_y)

def preprocess_distributions(X):
    """Apply log-scaling and normalization"""
    processed = []
    for dist in X:
        dist = dist.astype(np.float32)
        # Replace zeros
        min_nonzero = dist[dist > 0].min() if np.any(dist > 0) else 1e-10
        dist[dist == 0] = min_nonzero
        # Log scale
        dist = np.log10(dist)
        # Normalize 0–1
        dist = (dist - dist.min()) / (dist.max() - dist.min() + 1e-10)
        processed.append(dist)
    return np.array(processed, dtype=np.float32)

def build_mms_cnn(input_shape=(32,16,32,1), num_classes=4):
    """CNN from Olshevsky et al. (2021)"""
    from tensorflow.keras import layers, models
    model = models.Sequential([
        layers.Conv3D(32, (5,3,5), strides=(2,1,2), activation="relu", input_shape=input_shape),
        layers.Conv3D(64, (3,3,3), strides=(1,1,1), activation="relu"),
        layers.MaxPooling3D(pool_size=(2,2,2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-6),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# -------------------------------
# Generator
# -------------------------------

def cdf_generator(cdf_files, batch_size=32, num_classes=4):
    """Yield batches of (X, y) from CDF files"""
    while True:  # Infinite loop for Keras
        for file in cdf_files:
            cdf_path = os.path.join(data_dir, file)
            try:
                cdf = cdflib.CDF(cdf_path)

                raw_data = cdf.varget("mms1_dis_dist_fast")
                raw_epochs = cdf.varget("epoch")
                label_epochs = cdf.varget("label_epoch")
                labels = cdf.varget("label")

                # Match epochs
                X, y = match_epochs(raw_data, raw_epochs, label_epochs, labels)

                # Remove Unknowns (-1)
                mask = y >= 0
                X, y = X[mask], y[mask]

                # Preprocess
                X = preprocess_distributions(X)
                X = X[..., np.newaxis]  # add channel dim
                y = tf.keras.utils.to_categorical(y, num_classes=num_classes)

                # Yield batches
                for i in range(0, len(X), batch_size):
                    yield X[i:i+batch_size], y[i:i+batch_size]

            except Exception as e:
                print(f"⚠️ Skipping {file} due to error: {e}")
                continue

# -------------------------------
# Main Training
# -------------------------------

# Path to your labeled CDF directory
data_dir = r"D:\mms\Data\mms\mms1\fpi\fast\l2\dis-dist\2017\11\labeled"
cdf_files = [f for f in os.listdir(data_dir) if f.endswith(".cdf")]
print(f"Found {len(cdf_files)} CDF files")

# Build dataset from generator
batch_size = 32
train_dataset = tf.data.Dataset.from_generator(
    lambda: cdf_generator(cdf_files, batch_size=batch_size),
    output_signature=(
        tf.TensorSpec(shape=(None, 32, 16, 32, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 4), dtype=tf.float32)
    )
).prefetch(1)

# Build model
model = build_mms_cnn()

# Callbacks
callbacks = [
    EarlyStopping(monitor="loss", patience=5, restore_best_weights=True),
    ModelCheckpoint("mms_cnn_best_model.h5", save_best_only=True)
]

# Train model
steps_per_epoch = 50   # adjust depending on dataset size
epochs = 50

history = model.fit(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    callbacks=callbacks,
    verbose=2
)

# Save final model
model.save("mms_cnn_final_model.h5")
print("✅ Training complete, model saved as mms_cnn_final_model.h5")


In [ ]:
# -------------------------------
# Your helper functions
# -------------------------------

def match_epochs(raw_data, raw_epochs, label_epochs, labels):
    """Match raw_data samples with label epochs"""
    # Assuming raw_epochs and label_epochs are aligned or nearly aligned
    # Nearest-neighbor match
    matched_X, matched_y = [], []
    raw_epochs = np.array(raw_epochs)
    for le, lab in zip(label_epochs, labels):
        idx = np.argmin(np.abs(raw_epochs - le))
        matched_X.append(raw_data[idx])
        matched_y.append(lab)
    return np.array(matched_X), np.array(matched_y)

def preprocess_distributions(X):
    """Apply log-scaling and normalization"""
    processed = []
    for dist in X:
        dist = dist.astype(np.float32)
        # Replace zeros
        min_nonzero = dist[dist > 0].min() if np.any(dist > 0) else 1e-10
        dist[dist == 0] = min_nonzero
        # Log scale
        dist = np.log10(dist)
        # Normalize 0–1
        dist = (dist - dist.min()) / (dist.max() - dist.min() + 1e-10)
        processed.append(dist)
    return np.array(processed, dtype=np.float32)

def build_mms_cnn(input_shape=(32,16,32,1), num_classes=4):
    """CNN from Olshevsky et al. (2021)"""
    from tensorflow.keras import layers, models
    model = models.Sequential([
        layers.Conv3D(32, (5,3,5), strides=(2,1,2), activation="relu", input_shape=input_shape),
        layers.Conv3D(64, (3,3,3), strides=(1,1,1), activation="relu"),
        layers.MaxPooling3D(pool_size=(2,2,2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-6),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# -------------------------------
# Sequential Training
# -------------------------------

# Path to your data directory
data_dir = r"D:\mms\Data\mms\mms1\fpi\fast\l2\dis-dist\2017\11\labeled"
all_cdf_files = [f for f in os.listdir(data_dir) if f.endswith(".cdf")]
print(f"Found {len(all_cdf_files)} CDF files")

# Extract time IDs from filenames
time_ids = []
for file in all_cdf_files:
    time_id = file.split('_')[-1].split('.')[0]
    time_ids.append(time_id)
    print(f"File: {file} -> Time ID: {time_id}")

# Build model once (will be reused for all files)
model = build_mms_cnn()
print("Model created with architecture:")
model.summary()

# Callbacks
callbacks = [
    EarlyStopping(monitor="loss", patience=3, restore_best_weights=True),
    ModelCheckpoint("mms_cnn_checkpoint.h5", save_best_only=True)
]

# Process each file sequentially
for i, file in enumerate(all_cdf_files):
    print(f"\n[{i+1}/{len(all_cdf_files)}] Training on file: {file}")
    
    try:
        # Load CDF file
        cdf_path = os.path.join(data_dir, file)
        cdf = cdflib.CDF(cdf_path)
        
        # Get data
        raw_data = cdf.varget("mms1_dis_dist_fast")
        raw_epochs = cdf.varget("epoch")
        
        # Check if this is a labeled file
        try:
            label_epochs = cdf.varget("label_epoch")
            labels = cdf.varget("label")
            
            # Match epochs and process using the improved function
            X, y = match_epochs(raw_data, raw_epochs, label_epochs, labels)
            
            # Remove Unknowns (-1)
            mask = y >= 0
            X, y = X[mask], y[mask]
            
            if len(X) == 0:
                print(f"No valid data points in {file}, skipping")
                continue
                
            # Preprocess using the improved function
            X = preprocess_distributions(X)
            X = X[..., np.newaxis]  # Add channel dimension
            y = to_categorical(y, num_classes=4)
            
            print(f"Training on {len(X)} samples from {file}")
            
            # Train for a few epochs on this file
            history = model.fit(
                X, y,
                batch_size=32,
                epochs=100,  # Few epochs per file
                callbacks=callbacks,
                verbose=1
            )
            
            # Save intermediate model after each file
            model.save(f"mms_cnn_after_file_{i+1}.h5")
            
        except Exception as e:
            print(f"Could not find labels in {file} or other error: {e}")
            continue
            
    except Exception as e:
        print(f"Error processing {file}: {e}")
        continue

# Save final model after training on all files
model.save("mms_cnn_final_model.h5")
print("✅ Sequential training complete, final model saved as mms_cnn_final_model.h5")

50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.0000e+00 - loss: 1.9188
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.1353 - loss: 1.1885Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.1353 - loss: 1.1885

50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.4581 - loss: 1.0260
Epoch 3/100
 8/50 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 1.0000 - loss: 0.7291Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 1.0000 - loss: 0.6581

50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 1.0000 - loss: 0.5847



[38/194] Training on file: mms1_fpi_fast_l2_dis-dist_20171105040000_v3.4.0.cdf
Training on 1600 samples from mms1_fpi_fast_l2_dis-dist_20171105040000_v3.4.0.cdf
Epoch 1/100
37/50 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.0177 - loss: 1.2840Training on 1600 samples from mms1_fpi_fast_l2_dis-dist_20171105040000_v3.4.0.cdf
Epoch 1/100
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.0794 - loss: 1.2427

50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.3688 - loss: 1.0740
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 1.0000 - loss: 0.6607Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 1.0000 - loss: 0.6607

...


[194/194] Training on file: mms1_fpi_fast_l2_dis-dist_20171130220000_v3.4.0.cdf
Training on 800 samples from mms1_fpi_fast_l2_dis-dist_20171130220000_v3.4.0.cdf
Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 1.0000 - loss: 0.0401Training on 800 samples from mms1_fpi_fast_l2_dis-dist_20171130220000_v3.4.0.cdf
Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 1.0000 - loss: 0.0401

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 1.0000 - loss: 0.0369
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 1.0000 - loss: 0.0341Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 1.0000 - loss: 0.0341

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 1.0000 - loss: 0.0294
Epoch 3/100
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 1.0000 - loss: 0.0256Epoch 3/100
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 1.0000 - loss: 0.0256

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 1.0000 - loss: 0.0241


✅ Sequential training complete, final model saved as mms_cnn_final_model.h5
